In [1]:
# setup
import pandas as pd
import numpy as np

In [2]:
# carrega o dataset 'brasileirao_2014_2024.csv' (extraído da basedosdados.org)
df_br = pd.read_csv('../data/brasileirao_2014_2024.csv', index_col=0)

# exibe informações sobre o dataframe: tipo das variáveis e contagem de valores ausentes
df_br.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4086 entries, 0 to 4085
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ano_campeonato          4086 non-null   int64  
 1   rodada                  4086 non-null   int64  
 2   estadio                 4086 non-null   object 
 3   publico                 4085 non-null   float64
 4   publico_max             4033 non-null   float64
 5   arbitro                 4086 non-null   object 
 6   time_mandante           4086 non-null   object 
 7   gols_mandante           4085 non-null   float64
 8   time_visitante          4086 non-null   object 
 9   gols_visitante          4085 non-null   float64
 10  escanteios_mandante     2090 non-null   float64
 11  escanteios_visitante    2090 non-null   float64
 12  faltas_mandante         2090 non-null   float64
 13  faltas_visitante        2090 non-null   float64
 14  impedimentos_mandante   2090 non-null   float

In [3]:
# verifica se todas as rodadas estão completas (por ano de campeonato)
# 20 times, 10 jogos/rodada, 38 rodadas
rodadas_ano = (
    df_br.groupby(['ano_campeonato', 'rodada'])
    .size()
    .reset_index(name='partidas')
    .pivot(index='ano_campeonato', columns='rodada', values='partidas')
)

(rodadas_ano == 10).all(axis=1)

ano_campeonato
2014     True
2015     True
2016     True
2017     True
2018     True
2019     True
2020     True
2021     True
2022     True
2023     True
2024    False
dtype: bool

In [4]:
# remove os dados incompletos de 2024 e exibe informações do dataframe novamente
df_br = df_br.query('ano_campeonato < 2024')
df_br.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3800 entries, 0 to 4085
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ano_campeonato          3800 non-null   int64  
 1   rodada                  3800 non-null   int64  
 2   estadio                 3800 non-null   object 
 3   publico                 3799 non-null   float64
 4   publico_max             3756 non-null   float64
 5   arbitro                 3800 non-null   object 
 6   time_mandante           3800 non-null   object 
 7   gols_mandante           3799 non-null   float64
 8   time_visitante          3800 non-null   object 
 9   gols_visitante          3799 non-null   float64
 10  escanteios_mandante     1879 non-null   float64
 11  escanteios_visitante    1879 non-null   float64
 12  faltas_mandante         1879 non-null   float64
 13  faltas_visitante        1879 non-null   float64
 14  impedimentos_mandante   1879 non-null   float

In [5]:
# checando único valor ausente nas colunas 'publico', 'gols_mandante' e 'gols_visitante'
df_br.query('publico.isna()')

,ano_campeonato,rodada,estadio,publico,publico_max,arbitro,time_mandante,gols_mandante,time_visitante,gols_visitante,escanteios_mandante,escanteios_visitante,faltas_mandante,faltas_visitante,impedimentos_mandante,impedimentos_visitante
3331,2016,38,Arena Condá,NaN,22600.0,Rodrigo D'Alonso Ferreira,Chapecoense,NaN,Atlético-MG,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# salva o índice da partida identificada acima: Chapecoense, 38ª rodada, 2016
# WO definido por conta do acidente áereo sofrido pelo time da Chape
idx = df_br.query('ano_campeonato == 2016 & rodada == 38 & time_mandante == "Chapecoense"').index

# seleciona todas as colunas numéricas, exceto 'ano_campeonato', 'rodada' e 'publico_max'
cols = df_br.select_dtypes(include='number').columns.difference(['ano_campeonato', 'rodada', 'publico_max'])

# atribui zero às variáveis numéricas selecionadas para o jogo afetado pelo WO
df_br.loc[idx, cols] = 0

# confirma se as alterações foram feitas para o jogo específico
df_br.query('ano_campeonato == 2016 & rodada == 38 & time_mandante == "Chapecoense"')

,ano_campeonato,rodada,estadio,publico,publico_max,arbitro,time_mandante,gols_mandante,time_visitante,gols_visitante,escanteios_mandante,escanteios_visitante,faltas_mandante,faltas_visitante,impedimentos_mandante,impedimentos_visitante
3331,2016,38,Arena Condá,0.0,22600.0,Rodrigo D'Alonso Ferreira,Chapecoense,0.0,Atlético-MG,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# correção de valores ausentes na variável 'publico_max': cálculo da moda agrupado por 'estadio'
estadio_mode = (
    df_br
    .groupby('estadio')['publico_max']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
)

# preenche valores nulos em 'publico_max'
df_br['publico_max'] = df_br['publico_max'].fillna(df_br['estadio'].map(estadio_mode))

# exibe novamente informações sobre o dataframe
df_br.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3800 entries, 0 to 4085
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ano_campeonato          3800 non-null   int64  
 1   rodada                  3800 non-null   int64  
 2   estadio                 3800 non-null   object 
 3   publico                 3800 non-null   float64
 4   publico_max             3798 non-null   float64
 5   arbitro                 3800 non-null   object 
 6   time_mandante           3800 non-null   object 
 7   gols_mandante           3800 non-null   float64
 8   time_visitante          3800 non-null   object 
 9   gols_visitante          3800 non-null   float64
 10  escanteios_mandante     1880 non-null   float64
 11  escanteios_visitante    1880 non-null   float64
 12  faltas_mandante         1880 non-null   float64
 13  faltas_visitante        1880 non-null   float64
 14  impedimentos_mandante   1880 non-null   float

In [8]:
# checa valores ausentes restantes na variável 'publico_max':
# Atlético-MG, 7ª rodada, 2014
# Goiás, 27ª rodada, 2023
df_br.query('publico_max.isna()')

,ano_campeonato,rodada,estadio,publico,publico_max,arbitro,time_mandante,gols_mandante,time_visitante,gols_visitante,escanteios_mandante,escanteios_visitante,faltas_mandante,faltas_visitante,impedimentos_mandante,impedimentos_visitante
147,2014,7,Estádio Municipal João Lamego Netto,15668.0,NaN,Wagner do Nascimento Magalhães,Atlético-MG,0.0,Criciúma EC,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2190,2023,27,Estádio de Hailé Pinheiro,12685.0,NaN,Rodrigo Jose Pereira de Lima,Goiás,2.0,São Paulo,0.0,6.0,3.0,18.0,15.0,0.0,0.0


In [9]:
# calcula a moda de 'publico_max' para jogos em que o Atlético-MG foi o mandante em 2014
atletico_mode = (
    df_br
    .query('ano_campeonato == 2014')
    .loc[df_br['time_mandante'] == 'Atlético-MG', 'publico_max']
    .mode()
    .iloc[0]
)

# obtem o índice do jogo específico
idx = df_br.query('ano_campeonato == 2014 & rodada == 7 & time_mandante == "Atlético-MG"').index

# atualiza o valor de 'publico_max' com a moda calculada
df_br.loc[idx, 'publico_max'] = atletico_mode

# re-exibe os detalhes do jogo atualizado
df_br.query('ano_campeonato == 2014 & rodada == 7 & time_mandante == "Atlético-MG"')

,ano_campeonato,rodada,estadio,publico,publico_max,arbitro,time_mandante,gols_mandante,time_visitante,gols_visitante,escanteios_mandante,escanteios_visitante,faltas_mandante,faltas_visitante,impedimentos_mandante,impedimentos_visitante
147,2014,7,Estádio Municipal João Lamego Netto,15668.0,15000.0,Wagner do Nascimento Magalhães,Atlético-MG,0.0,Criciúma EC,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# calcula a moda de 'publico_max' para jogos em que o Goiás foi o mandante em 2023
goias_mode = (
    df_br
    .query('ano_campeonato == 2023')
    .loc[df_br['time_mandante'] == 'Goiás', 'publico_max']
    .mode()
    .iloc[0]
)

# obtem o índice do jogo específico
idx = df_br.query('ano_campeonato == 2023 & rodada == 27 & time_mandante == "Goiás"').index

# atualiza o valor de 'publico_max' com a moda calculada
df_br.loc[idx, 'publico_max'] = goias_mode

# re-exibe os detalhes do jogo atualizado
df_br.query('ano_campeonato == 2023 & rodada == 27 & time_mandante == "Goiás"')

,ano_campeonato,rodada,estadio,publico,publico_max,arbitro,time_mandante,gols_mandante,time_visitante,gols_visitante,escanteios_mandante,escanteios_visitante,faltas_mandante,faltas_visitante,impedimentos_mandante,impedimentos_visitante
2190,2023,27,Estádio de Hailé Pinheiro,12685.0,14525.0,Rodrigo Jose Pereira de Lima,Goiás,2.0,São Paulo,0.0,6.0,3.0,18.0,15.0,0.0,0.0


In [11]:
# exibe novamente informações sobre o dataframe para verificar missings restantes
df_br.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3800 entries, 0 to 4085
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ano_campeonato          3800 non-null   int64  
 1   rodada                  3800 non-null   int64  
 2   estadio                 3800 non-null   object 
 3   publico                 3800 non-null   float64
 4   publico_max             3800 non-null   float64
 5   arbitro                 3800 non-null   object 
 6   time_mandante           3800 non-null   object 
 7   gols_mandante           3800 non-null   float64
 8   time_visitante          3800 non-null   object 
 9   gols_visitante          3800 non-null   float64
 10  escanteios_mandante     1880 non-null   float64
 11  escanteios_visitante    1880 non-null   float64
 12  faltas_mandante         1880 non-null   float64
 13  faltas_visitante        1880 non-null   float64
 14  impedimentos_mandante   1880 non-null   float

In [12]:
# vetor de colunas que ainda possuem valores ausentes
cols = ['escanteios_mandante', 'escanteios_visitante', 'faltas_mandante', 'faltas_visitante', 'impedimentos_mandante', 'impedimentos_visitante']

# total de valores ausentes por coluna
missing_total = df_br[cols].isna().sum()

# linhas com valores ausentes simultaneamente em todas as colunas especificadas
missing_simultaneo = df_br[cols].isna().all(axis=1).sum()

# printa o total de valores ausentes por coluna
print('Total de missings por coluna:')
print(missing_total)

# printa o total de linhas com valores ausentes simultâneos:
# restaram apenas jogos que não tem nenhum tipo de informação, provavelmente por problemas nos dados
print('\nTotal de missings simultâneos:', missing_simultaneo)

# checa se os valores ausentes, além de simultâneos, estão concentrados em anos específicos
# dá pra observar que os dados passam a ser corretamente preenchidos somente a partir de 2020, com alguma melhoria já a partir de 2018
# não tem muito o que fazer nesse caso, vamos só ignorar :)
(
    df_br
    .groupby('ano_campeonato')[cols]
    .apply(lambda x: x.isnull().sum())
    .sort_values('ano_campeonato')
)

Total de missings por coluna:
escanteios_mandante       1920
escanteios_visitante      1920
faltas_mandante           1920
faltas_visitante          1920
impedimentos_mandante     1920
impedimentos_visitante    1920
dtype: int64

Total de missings simultâneos: 1920


,escanteios_mandante,escanteios_visitante,faltas_mandante,faltas_visitante,impedimentos_mandante,impedimentos_visitante
ano_campeonato,,,,,,
2014,380,380,380,380,380,380
2015,380,380,380,380,380,380
2016,379,379,379,379,379,379
2017,380,380,380,380,380,380
2018,169,169,169,169,169,169
2019,212,212,212,212,212,212
2020,6,6,6,6,6,6
2021,5,5,5,5,5,5
2022,7,7,7,7,7,7


In [13]:
# exporta novo CSV com os dados limpos
df_br.to_csv('../data/brasileirao_2014_2024_clean.csv')